In [1]:
import pandas as pd
import googlemaps
import time
from bs4 import BeautifulSoup as bs
import requests as req

In [4]:
# 設定目標 URL
# url = 'https://en.wikipedia.org/wiki/City_of_Brisbane'
url = 'https://en.wikipedia.org/wiki/Logan_City'
# url = 'https://en.wikipedia.org/wiki/Redland_City'

# 初始化空的 DataFrame
final_city_table = pd.DataFrame(columns=['towns'])

# 發送 HTTP 請求
response = req.get(url)
soup = bs(response.text, 'lxml')

# towns = [li.get_text() for li in soup.select("#mw-content-text > div.mw-content-ltr.mw-parser-output > div > ul > li")]
towns = [li.get_text() for li in soup.select("#mw-content-text > div.mw-content-ltr.mw-parser-output > div:nth-child(78) > table > tbody > tr > td > ul > li")]
# towns = [li.get_text() for li in soup.select("#mw-content-text > div.mw-content-ltr.mw-parser-output > div:nth-child(86) > table > tbody > tr > td > ul > li")]




# 將抓取到的城鎮名稱存入 DataFrame
final_city_table['towns'] = towns

# 移除有缺失值的行
final_city_table = final_city_table.dropna()

# 設定存儲路徑
path = "C:/Users/11021041/web_scraping/Google/AU/"

# 將結果保存為 Excel 文件
final_city_table.to_excel(path + "澳洲_LoganCity_城鎮名稱_20240913.xlsx", index=False)

In [35]:
# 公司key
gmaps_key = "GOOGLE MAPS API KEY"
# 或使用你的 Google Maps API 金鑰

# 設定工作目錄
import os
os.chdir("C:/Users/11021041/web_scraping/Google/AU/直客")

In [4]:
# # 讀取城市名稱資料
# final_city_birsbane = pd.read_excel("../澳洲_Brisbane_城鎮名稱_20240913.xlsx")

# # final_city_redland = pd.read_excel("../澳洲_RedlandCity_城鎮名稱_20240913.xlsx")

# # final_city_logan = pd.read_excel("../澳洲_LoganCity_城鎮名稱_20240913.xlsx")

# final_city = pd.concat([final_city_birsbane, final_city_redland, final_city_logan], axis=0)

final_city = pd.read_excel("../Brisbane_suburbs.xlsx")

selected_cities = final_city[['Suburb', 'City']]

print(selected_cities)

               Suburb      City
0        Acacia Ridge  Brisbane
1        Albany Creek  Brisbane
2              Albion  Brisbane
3            Alderley  Brisbane
4     Alexandra Hills  Brisbane
..                ...       ...
527           Sheldon   Redland
528        Thorneside   Redland
529        Thornlands   Redland
530    Victoria Point   Redland
531  Wellington Point   Redland

[532 rows x 2 columns]


In [5]:
# 搜尋關鍵字(直客)
# search_vec = ["Interior Design", "Renovation","Interior Design", "building material supplier", "Design and Build", "laminate distributor",
#               "Architect", "interior material", "system furniture", "cabinet", "Contractor", "carpentry", "furniture producer", "joinery"]

# search_vec = ["Architect", "Interior Design", "Renovation", "Design and Build", "Contractor", "carpentry", "cabinet", "joinery"]

search_vec = ["Interior Design", "Renovation"]

final_search = pd.DataFrame()
country = "Australia"
for i in search_vec:
    search_words = i
    city_search = pd.DataFrame(selected_cities, columns=["Suburb","City"])
    city_search['search_combine_words'] = search_words + ', ' + city_search['Suburb'] + ", " + city_search['City'] +", " + country
    city_search['completion'] = ""
    city_search['keywords'] = i

    final_search = pd.concat([final_search, city_search])

final_search = final_search.dropna()

final_search

,Suburb,City,search_combine_words,completion,keywords
0,Acacia Ridge,Brisbane,"Interior Design, Acacia Ridge, Brisbane, Austr...",,Interior Design
1,Albany Creek,Brisbane,"Interior Design, Albany Creek, Brisbane, Austr...",,Interior Design
2,Albion,Brisbane,"Interior Design, Albion, Brisbane, Australia",,Interior Design
3,Alderley,Brisbane,"Interior Design, Alderley, Brisbane, Australia",,Interior Design
4,Alexandra Hills,Brisbane,"Interior Design, Alexandra Hills, Brisbane, Au...",,Interior Design
...,...,...,...,...,...
527,Sheldon,Redland,"Renovation, Sheldon, Redland, Australia",,Renovation
528,Thorneside,Redland,"Renovation, Thorneside, Redland, Australia",,Renovation
529,Thornlands,Redland,"Renovation, Thornlands, Redland, Australia",,Renovation
530,Victoria Point,Redland,"Renovation, Victoria Point, Redland, Australia",,Renovation


In [6]:
# Google Maps客户端初始化
gmaps = googlemaps.Client(key=gmaps_key)

final_data = pd.DataFrame()

for i in range(len(final_search)):
    print(f"Processing search {i} of {len(final_search)}")

    # 建立搜索字符串
    search_string = final_search.iloc[i]['search_combine_words']

    # 调用 Google Places API
    gplaces = gmaps.places(search_string, radius=5000, language='en')  # 搜索半径5000米

    if gplaces['status'] != 'OK':
        print(gplaces['status'])
        print(i)
        continue

    # # 主要城市則多抓一些商家信息
    # if final_search.iloc[i]['town'] == 'Qazigund':
    #     for _ in range(3):  # 多抓取三次
    #         token = gplaces.get('next_page_token')
    #         if not token:
    #             break
    #         time.sleep(3)
    #         gplaces = gmaps.places(search_string, page_token=token, language='en')

    map_detail_data = pd.json_normalize(gplaces['results'])

    columns_to_remove = ["photos", "icon", "plus_code", "geometry", "opening_hours"]
    for column in columns_to_remove:
        map_detail_data = map_detail_data.loc[:, ~map_detail_data.columns.str.contains(column)]

    map_detail_data['keywords'] = final_search.iloc[i]['keywords']
    final_data = pd.concat([final_data, map_detail_data], ignore_index=True)

    final_search.loc[i, 'completion'] = "Y"

print(final_data)


Processing search 0 of 1064
Processing search 1 of 1064
Processing search 2 of 1064
Processing search 3 of 1064
Processing search 4 of 1064
ZERO_RESULTS
4
Processing search 5 of 1064
Processing search 6 of 1064
ZERO_RESULTS
6
Processing search 7 of 1064
Processing search 8 of 1064
ZERO_RESULTS
8
Processing search 9 of 1064
Processing search 10 of 1064
Processing search 11 of 1064
Processing search 12 of 1064
Processing search 13 of 1064
Processing search 14 of 1064
Processing search 15 of 1064
ZERO_RESULTS
15
Processing search 16 of 1064
Processing search 17 of 1064
Processing search 18 of 1064
Processing search 19 of 1064
Processing search 20 of 1064
Processing search 21 of 1064
Processing search 22 of 1064
Processing search 23 of 1064
ZERO_RESULTS
23
Processing search 24 of 1064
Processing search 25 of 1064
Processing search 26 of 1064
Processing search 27 of 1064
Processing search 28 of 1064
Processing search 29 of 1064
Processing search 30 of 1064
Processing search 31 of 1064
Proce

In [5]:
gmaps = googlemaps.Client(key=gmaps_key)

final_data = pd.DataFrame()

for i in range(len(final_search)):
    print(f"Processing search {i+1} of {len(final_search)}")

    # 建立搜索字符串
    search_string = final_search.iloc[i]['search_combine_words']

    # 調用 Google Places API
    gplaces = gmaps.places(search_string, radius=5000, language='en')  # 搜索半徑5000米

    if gplaces['status'] != 'OK':
        print(gplaces['status'])
        print(i)
        continue

    map_detail_data = pd.json_normalize(gplaces['results'])

    for _ in range(3):  # 多抓取三次
        token = gplaces.get('next_page_token')
        if not token:
            break
        time.sleep(3)
        gplaces = gmaps.places(search_string, page_token=token, language='en')
        
        # 合併新抓取的結果
        if gplaces['status'] == 'OK':
            new_results = pd.json_normalize(gplaces['results'])
            map_detail_data = pd.concat([map_detail_data, new_results], ignore_index=True)
        else:
            break

    # 刪除不需要的欄位
    columns_to_remove = ["photos", "icon", "plus_code", "geometry", "opening_hours"]
    map_detail_data = map_detail_data.loc[:, ~map_detail_data.columns.str.contains('|'.join(columns_to_remove))]

    # 添加關鍵詞欄位
    map_detail_data['keywords'] = final_search.iloc[i]['keywords']
    
    # 拼接結果
    final_data = pd.concat([final_data, map_detail_data], ignore_index=True)

    final_search.loc[i, 'completion'] = "Y"

print(final_data)

Processing search 1 of 546
Processing search 2 of 546
Processing search 3 of 546
Processing search 4 of 546
Processing search 5 of 546
Processing search 6 of 546
Processing search 7 of 546
Processing search 8 of 546
Processing search 9 of 546
Processing search 10 of 546
Processing search 11 of 546
Processing search 12 of 546
Processing search 13 of 546
Processing search 14 of 546
Processing search 15 of 546
Processing search 16 of 546
Processing search 17 of 546
Processing search 18 of 546
Processing search 19 of 546
Processing search 20 of 546
Processing search 21 of 546
Processing search 22 of 546
Processing search 23 of 546
Processing search 24 of 546
Processing search 25 of 546
Processing search 26 of 546
Processing search 27 of 546
Processing search 28 of 546
Processing search 29 of 546
Processing search 30 of 546
Processing search 31 of 546
Processing search 32 of 546
Processing search 33 of 546
Processing search 34 of 546
Processing search 35 of 546
Processing search 36 of 546
P

In [7]:
# 儲存到 Excel
final_data.to_excel("../AU_header_C&D_20250313_rawdata.xlsx", index=False)

In [8]:
# 列出所有文件
file_names = os.listdir()
print(file_names)
# file_names = [file for file in file_names if 'header' in file]
file_names = [file for file in file_names if 'data' in file]

data_list = []

# # 單一文件
# data = pd.read_excel('IN_data_excluded_20240812_all.xlsx')
# data_list.append(data)

# 逐一讀取文件並添加到 data_list (只取place_id進行合併)
for file in file_names:
    data = pd.read_excel(file)
    # 將 place_id 列提取出来，加入 data_list
    data_list.append(data)

# old_header = pd.concat(data_list, ignore_index=True)
old_place_ids = pd.concat([data['place_id'] for data in data_list], ignore_index=True)

# final_data = final_data[~final_data['place_id'].isin(old_header['place_id'])]
final_data = final_data[~final_data['place_id'].isin(old_place_ids)]

['AU_data_excluded_20230830.xlsx', 'AU_data_excluded_20230831.xlsx', 'AU_data_excluded_20230911.xlsx', 'AU_data_excluded_20240919(直客).xlsx', 'AU_data_excluded_20240919(直客)_final.xlsx', 'AU_data_excluded_end_cs_20230901.xlsx']


In [9]:
final_data = pd.DataFrame(final_data)
final_data

,business_status,formatted_address,name,place_id,rating,reference,types,user_ratings_total,keywords,permanently_closed,price_level
61,OPERATIONAL,"7 Ashgrove Ave, Ashgrove QLD 4060, Australia",4aHAUS Design Studio,ChIJm5GlEv1XkWsRvjOQPX3F3wA,5.0,ChIJm5GlEv1XkWsRvjOQPX3F3wA,"[general_contractor, point_of_interest, establ...",9.0,Interior Design,NaN,NaN
81,OPERATIONAL,"Unit 3/69 Secam St, Mansfield QLD 4122, Australia",Livin By Design,ChIJC60A2vVbkWsR8FQXnAfYvuw,5.0,ChIJC60A2vVbkWsR8FQXnAfYvuw,"[general_contractor, school, point_of_interest...",9.0,Interior Design,NaN,NaN
88,OPERATIONAL,"16 Tetra Cres, Ripley QLD 4306, Australia",Ellenbe Interior Design,ChIJObxk2iG1lmsRME1sAOpIod0,0.0,ChIJObxk2iG1lmsRME1sAOpIod0,"[general_contractor, point_of_interest, establ...",0.0,Interior Design,NaN,NaN
90,OPERATIONAL,"6 Amity St, New Farm QLD 4005, Australia",Fairweather Architecture,ChIJqzMcVCdakWsRAdIdJ_g149w,0.0,ChIJqzMcVCdakWsRAdIdJ_g149w,"[point_of_interest, establishment]",0.0,Interior Design,NaN,NaN
108,OPERATIONAL,"49 Corymbia Cres, Anstead QLD 4070, Australia",Centresquare,ChIJ14aDK2ho1moR-3SQwicF9Xc,4.7,ChIJ14aDK2ho1moR-3SQwicF9Xc,"[general_contractor, point_of_interest, establ...",24.0,Interior Design,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
14097,OPERATIONAL,"14 Tulloch Dr, Wellington Point QLD 4160, Aust...",Retro Roof Restorations Brisbane,ChIJBaoOlkdekWsRCHDVO3bWUj0,4.1,ChIJBaoOlkdekWsRCHDVO3bWUj0,"[roofing_contractor, point_of_interest, establ...",16.0,Renovation,NaN,NaN
14099,OPERATIONAL,"Unit 16/15 Peterson St, Wellington Point QLD 4...",Just Strip It,ChIJcyA2kMVhkWsRm6Aj1JOnhNQ,5.0,ChIJcyA2kMVhkWsRm6Aj1JOnhNQ,"[general_contractor, point_of_interest, establ...",2.0,Renovation,NaN,NaN
14101,OPERATIONAL,"405 Old Cleveland Rd E, Birkdale QLD 4159, Aus...",Vivid Projects Pty Ltd,ChIJdxQBQJFbkWsRGlBTGknHqEA,4.9,ChIJdxQBQJFbkWsRGlBTGknHqEA,"[painter, roofing_contractor, point_of_interes...",44.0,Renovation,NaN,NaN
14104,OPERATIONAL,"13 Manor Dr, Wellington Point QLD 4160, Australia",DK Plumbing | Plumber Wellington Point,ChIJj89fLc1gkWsRlxCr4BkCbEU,5.0,ChIJj89fLc1gkWsRlxCr4BkCbEU,"[plumber, point_of_interest, establishment]",29.0,Renovation,NaN,NaN


In [31]:
# exclude "name" key words
exclude = ["Noodle", "Cafe", "CAFE","Restaurant", "학과", "식당", "care", "케어", "real estate", 
             "부동산", "bakery", "빵집", "food", "음식", "hair", "Hair", "florist", "플로리스트", "dentist", "치과 의사",
             "bar", "술집", "beauty", "아름다움", "University", "대학교", "school", "School", "학교", "transit", "운송",
             "convience store", "편의점", "shoe", "구두", "movie", "영화", "lodging", "하숙", "worship", "예배", "TATTOO", "타투",
             "公園", "點心", "圖書館", "花園", "酒店", "餅店", "獸醫", "歷史館", "熟食", "燒肉", "藥房", "教堂", "Café",
             "Cartier", "Dior", "自修室", "牌坊", "酒店", "保險", "燒烤", "Subway", "大學", "郵政局", "美食", "焚化爐", "咖啡",
             "水電", "活動中心", "檳榔攤", "超商", "鐵工廠", "瓦斯爐", "political", "gym", "accounting", "airport", "hospital", "casino", "library", "art_gallery", "natural_feature", 
             "cemetery", "Plumbing", "bus_station", "courthouse", "spa", "salon", "meal_delivery", 
             "pharmacy", "dentist", "doctor", "travel_agency", "bar",
             "university", "food", "shopping_mall", "city_hall", "bakery", "real_estate_agency", "restaurant", 
             "meal_takeaway", "atm", "bank", "finance", "school", "museum", "pharmacy", "drugstore", 
             "church", "place_of_worship", "bicycle_store", "plumber", "parking", "tourist_attraction", 
             "funeral_home", "secondary_school", "jewelry_store", "post_office", "electrician", "convenience_store", 
             "mosque", "hindu_temple", "embassy", "train_station", "moving_company", "gas_station", "airport"
             , "local_government_office", "supermarket", "department_store", "laundry", "health", "campground", 
             "town_square", "transit_station", "electronics_store", "insurance_agency", "stadium", "police", 
             "route", "lodging", "book_store", "park", "storage", "butchery", "Fruit", "Hotel", " Jeweller",
             "Electrical", "Laundry", "Motor", "Rental", "Health" ,"Basketball", "Apartment", "Coffee", "Gymnasium", "Audio", "flooring", "painting"
              "Curtain", "farm", "accociation", "Web", "Graphic",
               'Tailor', 'Music', 'Sport', 'aircon', 'jewelry', 'Painting',  'Hardware', 'Battery', 'Electronics', 'Electric', 
               'Printer', 'Moto', 'Bicycle',  'Gaming', 'trading', 'Garage', 'Merchandise', 'Cellphone',
               'Dormitory', 'Showroom', 'STEEL', 'Shipyard', 'Landmark', 'Vlog', 'Pet', 'Holy Spirit', 'Badminton', 'welding', 
               'Pools', 'Market', 'Warehouse', 'Garden', 'Wheels', 'Drilling', 'WATCH REPAIR', 'hydrant', 'Alumimum', 'service center', 'swimming', 'Mansion', 'Realty', 'luxe',
               'painter']


final_data = final_data[~final_data['name'].str.contains('|'.join(exclude), case=False, na=False)]

# exclude "types" key words
exclude_types = ["political", "gym", "accounting", "airport", "hospital", "casino", "library", "art_gallery", "natural_feature",
                 "cemetery", "Plumbing", "bus_station", "courthouse", "spa", "salon", "meal_delivery",
                 "pharmacy", "dentist", "doctor", "travel_agency", "bar",
                 "university", "food", "shopping_mall", "city_hall", "bakery", "real_estate_agency", "restaurant",
                 "meal_takeaway", "atm", "bank", "finance", "school", "museum", "pharmacy", "drugstore",
                 "church", "place_of_worship", "bicycle_store", "plumber", "parking", "tourist_attraction",
                 "funeral_home", "secondary_school", "jewelry_store", "post_office", "electrician", "convenience_store",
                 "mosque", "hindu_temple", "embassy", "train_station", "moving_company", "gas_station", "airport"
                 , "local_government_office", "supermarket", "department_store", "laundry", "health", "campground",
                 "town_square", "transit_station", "electronics_store", "insurance_agency", "stadium", "police",
                 "route", "lodging", "book_store", "park", "storage", "street_address", "clothing", "political", "fire_station",'car','pet']

final_data['types'] = final_data['types'].astype(str)
final_data['types'] = final_data['types'].str.strip()  
final_data = final_data[~final_data['types'].str.contains('|'.join(exclude_types), case=False, na=False)]

# 移除營業狀態為'CLOSED'的資料
final_data = final_data[~final_data['business_status'].astype(str).str.contains('closed', case=False, na=False) & final_data['business_status'].notna()]



C:\Users\11021041\AppData\Local\Temp\ipykernel_17636\2258793793.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data['types'] = final_data['types'].astype(str)
C:\Users\11021041\AppData\Local\Temp\ipykernel_17636\2258793793.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data['types'] = final_data['types'].str.strip()


In [32]:
# 移除重複的place_id
final_data.drop_duplicates(subset='place_id', inplace=True)
final_data

,place_id,name,formatted_address,business_status,rating,reference,types,user_ratings_total,keywords,permanently_closed,price_level
0,ChIJm5GlEv1XkWsRvjOQPX3F3wA,4aHAUS Design Studio,"7 Ashgrove Ave, Ashgrove QLD 4060, Australia",OPERATIONAL,5.0,ChIJm5GlEv1XkWsRvjOQPX3F3wA,"['general_contractor', 'point_of_interest', 'e...",9.0,Interior Design,NaN,NaN
1,ChIJObxk2iG1lmsRME1sAOpIod0,Ellenbe Interior Design,"16 Tetra Cres, Ripley QLD 4306, Australia",OPERATIONAL,0.0,ChIJObxk2iG1lmsRME1sAOpIod0,"['general_contractor', 'point_of_interest', 'e...",0.0,Interior Design,NaN,NaN
2,ChIJqzMcVCdakWsRAdIdJ_g149w,Fairweather Architecture,"6 Amity St, New Farm QLD 4005, Australia",OPERATIONAL,0.0,ChIJqzMcVCdakWsRAdIdJ_g149w,"['point_of_interest', 'establishment']",0.0,Interior Design,NaN,NaN
3,ChIJ14aDK2ho1moR-3SQwicF9Xc,Centresquare,"49 Corymbia Cres, Anstead QLD 4070, Australia",OPERATIONAL,4.7,ChIJ14aDK2ho1moR-3SQwicF9Xc,"['general_contractor', 'point_of_interest', 'e...",24.0,Interior Design,NaN,NaN
4,ChIJteXpk-9gwAoREQxxEnjZfkw,Janelle Mead Interiors,"2/830 Old Cleveland Rd, Carina QLD 4152, Austr...",OPERATIONAL,5.0,ChIJteXpk-9gwAoREQxxEnjZfkw,"['general_contractor', 'point_of_interest', 'e...",1.0,Interior Design,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
892,ChIJ7WXV0_1vkWsRdE6umZBL-TA,"Russell Island Timber, Decking & Cladding","14 Trimaran St, Russell Island QLD 4184, Austr...",OPERATIONAL,4.8,ChIJ7WXV0_1vkWsRdE6umZBL-TA,"['store', 'point_of_interest', 'establishment']",4.0,Renovation,NaN,NaN
893,ChIJpWn-YaBvkWsRhpIhTPEZa_s,FALCON CASTLE,"10 Falconhurst Rd, Russell Island QLD 4184, Au...",OPERATIONAL,5.0,ChIJpWn-YaBvkWsRhpIhTPEZa_s,"['general_contractor', 'point_of_interest', 'e...",1.0,Renovation,NaN,NaN
895,ChIJUeQfkYBDkWsRFvOCbb3YbS8,Brisco Roofing,"58 Belford Dr, Wellington Point QLD 4160, Aust...",OPERATIONAL,4.9,ChIJUeQfkYBDkWsRFvOCbb3YbS8,"['roofing_contractor', 'point_of_interest', 'e...",130.0,Renovation,NaN,NaN
896,ChIJcyA2kMVhkWsRm6Aj1JOnhNQ,Just Strip It,"Unit 16/15 Peterson St, Wellington Point QLD 4...",OPERATIONAL,5.0,ChIJcyA2kMVhkWsRm6Aj1JOnhNQ,"['general_contractor', 'point_of_interest', 'e...",2.0,Renovation,NaN,NaN


In [15]:
final_data = final_data[final_data['formatted_address'].str.contains('Australia')]
final_data

,business_status,formatted_address,name,place_id,rating,reference,types,user_ratings_total,keywords,permanently_closed,price_level
61,OPERATIONAL,"7 Ashgrove Ave, Ashgrove QLD 4060, Australia",4aHAUS Design Studio,ChIJm5GlEv1XkWsRvjOQPX3F3wA,5.0,ChIJm5GlEv1XkWsRvjOQPX3F3wA,"['general_contractor', 'point_of_interest', 'e...",9.0,Interior Design,NaN,NaN
88,OPERATIONAL,"16 Tetra Cres, Ripley QLD 4306, Australia",Ellenbe Interior Design,ChIJObxk2iG1lmsRME1sAOpIod0,0.0,ChIJObxk2iG1lmsRME1sAOpIod0,"['general_contractor', 'point_of_interest', 'e...",0.0,Interior Design,NaN,NaN
90,OPERATIONAL,"6 Amity St, New Farm QLD 4005, Australia",Fairweather Architecture,ChIJqzMcVCdakWsRAdIdJ_g149w,0.0,ChIJqzMcVCdakWsRAdIdJ_g149w,"['point_of_interest', 'establishment']",0.0,Interior Design,NaN,NaN
108,OPERATIONAL,"49 Corymbia Cres, Anstead QLD 4070, Australia",Centresquare,ChIJ14aDK2ho1moR-3SQwicF9Xc,4.7,ChIJ14aDK2ho1moR-3SQwicF9Xc,"['general_contractor', 'point_of_interest', 'e...",24.0,Interior Design,NaN,NaN
121,OPERATIONAL,"2/830 Old Cleveland Rd, Carina QLD 4152, Austr...",Janelle Mead Interiors,ChIJteXpk-9gwAoREQxxEnjZfkw,5.0,ChIJteXpk-9gwAoREQxxEnjZfkw,"['general_contractor', 'point_of_interest', 'e...",1.0,Interior Design,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
14050,OPERATIONAL,"10 Falconhurst Rd, Russell Island QLD 4184, Au...",FALCON CASTLE,ChIJpWn-YaBvkWsRhpIhTPEZa_s,5.0,ChIJpWn-YaBvkWsRhpIhTPEZa_s,"['general_contractor', 'point_of_interest', 'e...",1.0,Renovation,NaN,NaN
14074,OPERATIONAL,"11 Sandune Pl, Thornlands QLD 4164, Australia",Redlands Coast Painters,ChIJ52e2959nkWsRwzvtjn5D-yE,5.0,ChIJ52e2959nkWsRwzvtjn5D-yE,"['painter', 'point_of_interest', 'establishment']",22.0,Renovation,NaN,NaN
14096,OPERATIONAL,"58 Belford Dr, Wellington Point QLD 4160, Aust...",Brisco Roofing,ChIJUeQfkYBDkWsRFvOCbb3YbS8,4.9,ChIJUeQfkYBDkWsRFvOCbb3YbS8,"['roofing_contractor', 'point_of_interest', 'e...",130.0,Renovation,NaN,NaN
14099,OPERATIONAL,"Unit 16/15 Peterson St, Wellington Point QLD 4...",Just Strip It,ChIJcyA2kMVhkWsRm6Aj1JOnhNQ,5.0,ChIJcyA2kMVhkWsRm6Aj1JOnhNQ,"['general_contractor', 'point_of_interest', 'e...",2.0,Renovation,NaN,NaN


In [16]:
import requests

# 取得 Token , 設置請求參數 (台灣)
url = "https://login-p10.xiaoshouyi.com/auc/oauth2/token"
headers = {
    "Content-Type": "application/x-www-form-urlencoded"
}
data = {
    "grant_type": "password",
    "client_id": "bc4896f9e3e4fad682f9fe60d5fbaa2e",
    "client_secret": "3407faf39a38b6821f18cdbe019a56e3",
    "redirect_uri": "https://api-p10.xiaoshouyi.com",
    "username": "11021041@twkd.com",
    "password": "Marcellin01PrRDrSdz"
}

# 發送 post 請求
response = requests.post(url, headers=headers, data=data)

if response.status_code == 200:
    print(response.json())
else:
    print("請求失敗, 狀態碼:", response.status_code)

content = response.json()
ac_token = content["access_token"]


print(ac_token)

#### Company #### 
url_2 = "https://api-p10.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type": "application/x-www-form-urlencoded"
}
queryLocator = ''
crm_data = pd.DataFrame()

while True:
    data = {
        "xoql": '''SELECT 
        dimDepart.departName as 資料區域群組,
        customItem198__c.name as 公司型態代號,
        accountCode__c as CompanyID,
        accountName as 公司全名,
        customItem197__c as 公司簡稱,
        customItem202__c as 公司地址,
        Phone__c as combined_phone,
        SAP_CompanyID__c as SAP公司代號,
        contactNumber__c as 主要聯絡人行動電話
        FROM account 
        WHERE dimDepart.departName LIKE '%AU%'
        ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = requests.post(url_2, headers=headers, data=data)
    exist = response.json()
    data = pd.DataFrame(exist["data"]["records"])
    crm_data = pd.concat([crm_data, data], ignore_index=True, sort=False)

    if not exist['queryLocator']:
        break
    queryLocator = exist['queryLocator']

crm_data

{'access_token': '390d1bbccc4432a66a8ddfc7d40cdb695854bd95045ac4ac6648d7f1ca9dd9f1.MjU0NDU2OTY4NDAzNzYxMA==9', 'token_type': 'Bearer', 'expires_in': 86399, 'scope': 'all prm crm', 'tenant_id': '2544569684037610', 'login_type': '', 'encryption_key': '', 'client_id': 'bc4896f9e3e4fad682f9fe60d5fbaa2e', 'passport_id': '3006921588936589', 'mobile_params': '', 'instance_uri': 'api-p10.xiaoshouyi.com'}
390d1bbccc4432a66a8ddfc7d40cdb695854bd95045ac4ac6648d7f1ca9dd9f1.MjU0NDU2OTY4NDAzNzYxMA==9


,資料區域群組,公司型態代號,CompanyID,公司全名,公司簡稱,公司地址,combined_phone,SAP公司代號,主要聯絡人行動電話
0,AU,IB,GAC00093798,Indent Tile Centre,Indent Tile Centre,"11 Grey Gum Rd, Taree NSW 2430, Australia",0265524688,,
1,AU,IB,GAC00093799,Northern Rivers Tiles & Mosaics,Northern Rivers Tile,"177 Keen St, Lismore NSW 2480, Australia",0266215232,,
2,AU,IB,GAC00093800,Color Craft Kitchens,Color Craft Kitchens,"125 Mount Darragh Rd, Pambula NSW 2549, Australia",0264957153,,
3,AU,IB,GAC00093801,Stone Ambassador Western Australia,Stone Ambassador,"24 Creative St, Wangara WA 6065, Australia",0863836558,,
4,AU,IB,GAC00093802,Cabinet Makers Geelong,Geelong,"7 hardwood walk, leopold, vic 3224",0488860284,,610420940514
...,...,...,...,...,...,...,...,...,...
463,AU-加盟,,GAC00166941,CRM,CRM,,,,
464,AU-加盟,N,GAC00167118,Cindy Song,Cindy Song,,,,
465,AU-加盟,N,GAC00167120,BhoomiPatel,BhoomiPatel,,,,
466,AU-加盟,,GAC00167128,p L,p L,,,,


In [17]:

import numpy as np

# 將空字串替換為 NaN
crm_data.replace('', np.nan, inplace=True)

# 將空字串替換為 NaN
crm_data.replace('-', np.nan, inplace=True)

# 將空字串替換為 NaN
crm_data['combined_phone'] = crm_data['combined_phone'].replace(r'0000', np.nan, regex=True)
crm_data.replace('0', np.nan, inplace=True)

# 如果combined_phone為空，則使用主要聯絡人行動電話
crm_data['combined_phone'].fillna(crm_data['主要聯絡人行動電話'], inplace=True)

# 去掉CRM_data中combined_phone中的標點符號和空格ˇ
crm_data['new_phone_num'] = crm_data['combined_phone'].str.replace(r'\D', '', regex=True)

# 將na補0
crm_data['new_phone_num'].fillna("0", inplace=True)



# 選擇需要的欄位
req_columns_final_merge = ["place_id", "name", "formatted_address"]
final_merge = final_data[req_columns_final_merge]
final_merge.columns = ["place_id", "name", "formatted_address"]

req_columns_crm_data = ["公司全名", "公司簡稱", "CompanyID", "公司地址"]
crm_data = crm_data[req_columns_crm_data]
crm_data.columns = ["name", "short_name", "CompanyID", "formatted_address"]

# name exact first
final_merge = final_merge[~final_merge['name'].isin(crm_data['name'])]

# exact address match
final_merge = final_merge[~final_merge['formatted_address'].isin(crm_data['formatted_address'])]

final_data = pd.merge(final_merge, final_data, on=['place_id', 'name', 'formatted_address'], how='left')

final_data

,place_id,name,formatted_address,business_status,rating,reference,types,user_ratings_total,keywords,permanently_closed,price_level
0,ChIJm5GlEv1XkWsRvjOQPX3F3wA,4aHAUS Design Studio,"7 Ashgrove Ave, Ashgrove QLD 4060, Australia",OPERATIONAL,5.0,ChIJm5GlEv1XkWsRvjOQPX3F3wA,"['general_contractor', 'point_of_interest', 'e...",9.0,Interior Design,NaN,NaN
1,ChIJObxk2iG1lmsRME1sAOpIod0,Ellenbe Interior Design,"16 Tetra Cres, Ripley QLD 4306, Australia",OPERATIONAL,0.0,ChIJObxk2iG1lmsRME1sAOpIod0,"['general_contractor', 'point_of_interest', 'e...",0.0,Interior Design,NaN,NaN
2,ChIJqzMcVCdakWsRAdIdJ_g149w,Fairweather Architecture,"6 Amity St, New Farm QLD 4005, Australia",OPERATIONAL,0.0,ChIJqzMcVCdakWsRAdIdJ_g149w,"['point_of_interest', 'establishment']",0.0,Interior Design,NaN,NaN
3,ChIJ14aDK2ho1moR-3SQwicF9Xc,Centresquare,"49 Corymbia Cres, Anstead QLD 4070, Australia",OPERATIONAL,4.7,ChIJ14aDK2ho1moR-3SQwicF9Xc,"['general_contractor', 'point_of_interest', 'e...",24.0,Interior Design,NaN,NaN
4,ChIJteXpk-9gwAoREQxxEnjZfkw,Janelle Mead Interiors,"2/830 Old Cleveland Rd, Carina QLD 4152, Austr...",OPERATIONAL,5.0,ChIJteXpk-9gwAoREQxxEnjZfkw,"['general_contractor', 'point_of_interest', 'e...",1.0,Interior Design,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
893,ChIJpWn-YaBvkWsRhpIhTPEZa_s,FALCON CASTLE,"10 Falconhurst Rd, Russell Island QLD 4184, Au...",OPERATIONAL,5.0,ChIJpWn-YaBvkWsRhpIhTPEZa_s,"['general_contractor', 'point_of_interest', 'e...",1.0,Renovation,NaN,NaN
894,ChIJ52e2959nkWsRwzvtjn5D-yE,Redlands Coast Painters,"11 Sandune Pl, Thornlands QLD 4164, Australia",OPERATIONAL,5.0,ChIJ52e2959nkWsRwzvtjn5D-yE,"['painter', 'point_of_interest', 'establishment']",22.0,Renovation,NaN,NaN
895,ChIJUeQfkYBDkWsRFvOCbb3YbS8,Brisco Roofing,"58 Belford Dr, Wellington Point QLD 4160, Aust...",OPERATIONAL,4.9,ChIJUeQfkYBDkWsRFvOCbb3YbS8,"['roofing_contractor', 'point_of_interest', 'e...",130.0,Renovation,NaN,NaN
896,ChIJcyA2kMVhkWsRm6Aj1JOnhNQ,Just Strip It,"Unit 16/15 Peterson St, Wellington Point QLD 4...",OPERATIONAL,5.0,ChIJcyA2kMVhkWsRm6Aj1JOnhNQ,"['general_contractor', 'point_of_interest', 'e...",2.0,Renovation,NaN,NaN


In [29]:
history_data = pd.read_excel('../澳洲-布里斯本_CRM_1553【交辦管理-海外】.xlsx')


# 使用正則表達式拆分成多個欄位
history_data[['Company Name', 'Phone Number', 'Address']] = history_data['執行內容說明二'].str.extract(
    r'Company Name:\s*(.*?)\nPhone Number:\s*(.*)\n(?:Address:\s*(.*?)\n)?'
)

history_data = history_data[['Company Name', 'Phone Number', 'Address']]

history_data = pd.DataFrame(history_data)

history_data

,Company Name,Phone Number,Address
0,Interior Design by Jo,+61 422 037 848,"9 Ludlow St, Chapel Hill QLD 4069, Australia"
1,Skyring Architects,+61 7 3367 8678,"78 MacGregor Terrace, Bardon QLD 4065, Australia"
2,Cabinets Online,+61 7 3806 1199,"22 Josephine St, Loganholme QLD 4129, Australia"
3,MJM Cabinet Making,+61 417 781 648,"Shed 4/22 Moonbi St, Brendale QLD 4500, Australia"
4,Linear Design + Construct,+61 404 713 679,"1 Dora St, Hendra QLD 4011, Australia"
...,...,...,...
1548,Adept Carpentry,0414453499,"Wynnum QLD 4718, Australia"
1549,Woodrats Joinery PTY Ltd.,0738234788,"4/160 Redland Bay Rd, Capalaba QLD 4157, Austr..."
1550,Phantom Shopfitting and Joinery Pty Ltd,0735503382,"3/147 Boniface St, Archerfield QLD 4108, Austr..."
1551,Colonial Restoration Supplies Pty Ltd,0732682669,"20 Curtin Ave W, Hamilton QLD 4007, Australia"


In [33]:
# name exact first
final_data = final_data[~final_data['name'].isin(history_data['Company Name'])]

# exact address match
final_data = final_data[~final_data['formatted_address'].isin(history_data['Address'])]

final_data

,place_id,name,formatted_address,business_status,rating,reference,types,user_ratings_total,keywords,permanently_closed,price_level
0,ChIJm5GlEv1XkWsRvjOQPX3F3wA,4aHAUS Design Studio,"7 Ashgrove Ave, Ashgrove QLD 4060, Australia",OPERATIONAL,5.0,ChIJm5GlEv1XkWsRvjOQPX3F3wA,"['general_contractor', 'point_of_interest', 'e...",9.0,Interior Design,NaN,NaN
1,ChIJObxk2iG1lmsRME1sAOpIod0,Ellenbe Interior Design,"16 Tetra Cres, Ripley QLD 4306, Australia",OPERATIONAL,0.0,ChIJObxk2iG1lmsRME1sAOpIod0,"['general_contractor', 'point_of_interest', 'e...",0.0,Interior Design,NaN,NaN
2,ChIJqzMcVCdakWsRAdIdJ_g149w,Fairweather Architecture,"6 Amity St, New Farm QLD 4005, Australia",OPERATIONAL,0.0,ChIJqzMcVCdakWsRAdIdJ_g149w,"['point_of_interest', 'establishment']",0.0,Interior Design,NaN,NaN
3,ChIJ14aDK2ho1moR-3SQwicF9Xc,Centresquare,"49 Corymbia Cres, Anstead QLD 4070, Australia",OPERATIONAL,4.7,ChIJ14aDK2ho1moR-3SQwicF9Xc,"['general_contractor', 'point_of_interest', 'e...",24.0,Interior Design,NaN,NaN
4,ChIJteXpk-9gwAoREQxxEnjZfkw,Janelle Mead Interiors,"2/830 Old Cleveland Rd, Carina QLD 4152, Austr...",OPERATIONAL,5.0,ChIJteXpk-9gwAoREQxxEnjZfkw,"['general_contractor', 'point_of_interest', 'e...",1.0,Interior Design,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
892,ChIJ7WXV0_1vkWsRdE6umZBL-TA,"Russell Island Timber, Decking & Cladding","14 Trimaran St, Russell Island QLD 4184, Austr...",OPERATIONAL,4.8,ChIJ7WXV0_1vkWsRdE6umZBL-TA,"['store', 'point_of_interest', 'establishment']",4.0,Renovation,NaN,NaN
893,ChIJpWn-YaBvkWsRhpIhTPEZa_s,FALCON CASTLE,"10 Falconhurst Rd, Russell Island QLD 4184, Au...",OPERATIONAL,5.0,ChIJpWn-YaBvkWsRhpIhTPEZa_s,"['general_contractor', 'point_of_interest', 'e...",1.0,Renovation,NaN,NaN
895,ChIJUeQfkYBDkWsRFvOCbb3YbS8,Brisco Roofing,"58 Belford Dr, Wellington Point QLD 4160, Aust...",OPERATIONAL,4.9,ChIJUeQfkYBDkWsRFvOCbb3YbS8,"['roofing_contractor', 'point_of_interest', 'e...",130.0,Renovation,NaN,NaN
896,ChIJcyA2kMVhkWsRm6Aj1JOnhNQ,Just Strip It,"Unit 16/15 Peterson St, Wellington Point QLD 4...",OPERATIONAL,5.0,ChIJcyA2kMVhkWsRm6Aj1JOnhNQ,"['general_contractor', 'point_of_interest', 'e...",2.0,Renovation,NaN,NaN


In [ ]:
# 儲存到 Excel
final_data.to_excel("AU_header_C&D_20250313_excluded.xlsx", index=False)

#營業狀況[3]
#電話號碼[6]
#國際號碼[11]
#型態[20]
#公司網頁[25]

In [37]:
final_data = pd.read_excel('../AU_header_C&D_20250313_excluded_zips.xlsx',sheet_name='Sheet1')
final_data

,place_id,name,formatted_address,business_status,rating,reference,types,user_ratings_total,keywords,permanently_closed,price_level,Potential_Client
0,ChIJm5GlEv1XkWsRvjOQPX3F3wA,4aHAUS Design Studio,"7 Ashgrove Ave, Ashgrove QLD 4060, Australia",OPERATIONAL,5.0,ChIJm5GlEv1XkWsRvjOQPX3F3wA,"['general_contractor', 'point_of_interest', 'e...",9,Interior Design,NaN,NaN,Y
1,ChIJObxk2iG1lmsRME1sAOpIod0,Ellenbe Interior Design,"16 Tetra Cres, Ripley QLD 4306, Australia",OPERATIONAL,0.0,ChIJObxk2iG1lmsRME1sAOpIod0,"['general_contractor', 'point_of_interest', 'e...",0,Interior Design,NaN,NaN,NaN
2,ChIJqzMcVCdakWsRAdIdJ_g149w,Fairweather Architecture,"6 Amity St, New Farm QLD 4005, Australia",OPERATIONAL,0.0,ChIJqzMcVCdakWsRAdIdJ_g149w,"['point_of_interest', 'establishment']",0,Interior Design,NaN,NaN,NaN
3,ChIJ14aDK2ho1moR-3SQwicF9Xc,Centresquare,"49 Corymbia Cres, Anstead QLD 4070, Australia",OPERATIONAL,4.7,ChIJ14aDK2ho1moR-3SQwicF9Xc,"['general_contractor', 'point_of_interest', 'e...",24,Interior Design,NaN,NaN,NaN
4,ChIJteXpk-9gwAoREQxxEnjZfkw,Janelle Mead Interiors,"2/830 Old Cleveland Rd, Carina QLD 4152, Austr...",OPERATIONAL,5.0,ChIJteXpk-9gwAoREQxxEnjZfkw,"['general_contractor', 'point_of_interest', 'e...",1,Interior Design,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
582,ChIJ7WXV0_1vkWsRdE6umZBL-TA,"Russell Island Timber, Decking & Cladding","14 Trimaran St, Russell Island QLD 4184, Austr...",OPERATIONAL,4.8,ChIJ7WXV0_1vkWsRdE6umZBL-TA,"['store', 'point_of_interest', 'establishment']",4,Renovation,NaN,NaN,NaN
583,ChIJpWn-YaBvkWsRhpIhTPEZa_s,FALCON CASTLE,"10 Falconhurst Rd, Russell Island QLD 4184, Au...",OPERATIONAL,5.0,ChIJpWn-YaBvkWsRhpIhTPEZa_s,"['general_contractor', 'point_of_interest', 'e...",1,Renovation,NaN,NaN,NaN
584,ChIJUeQfkYBDkWsRFvOCbb3YbS8,Brisco Roofing,"58 Belford Dr, Wellington Point QLD 4160, Aust...",OPERATIONAL,4.9,ChIJUeQfkYBDkWsRFvOCbb3YbS8,"['roofing_contractor', 'point_of_interest', 'e...",130,Renovation,NaN,NaN,NaN
585,ChIJcyA2kMVhkWsRm6Aj1JOnhNQ,Just Strip It,"Unit 16/15 Peterson St, Wellington Point QLD 4...",OPERATIONAL,5.0,ChIJcyA2kMVhkWsRm6Aj1JOnhNQ,"['general_contractor', 'point_of_interest', 'e...",2,Renovation,NaN,NaN,NaN


In [38]:
# Initialize detail_data DataFrame
detail_data = pd.DataFrame(columns=[
    'place_id', 'formatted_phone_number', 'international_phone_number', 'website', 'types'
])

# Ensure the columns are of the correct data type
detail_data['place_id'] = detail_data['place_id'].astype(str)
detail_data['formatted_phone_number'] = detail_data['formatted_phone_number'].astype(str)
detail_data['international_phone_number'] = detail_data['international_phone_number'].astype(str)
detail_data['website'] = detail_data['website'].astype(str)
detail_data['types'] = detail_data['types'].astype(str)

detail_data

,place_id,formatted_phone_number,international_phone_number,website,types


In [39]:
# 取得去重後的place_id
detail_search = final_data['place_id'].unique()

In [40]:
# 遍歷所有place_id，依照place_id來取得商家資訊
for i in detail_search:
    print(f"Processing place_id {i} ({detail_search.tolist().index(i) + 1}/{len(detail_search)})")
    
    # 簡單檢查 place_id 是否有效
    if not i:
        print("Invalid place_id")
        continue

    # 调用Google Place Details API
    try:
        gplaces_detail = gmaps.place(place_id=i, language='en')
    except googlemaps.exceptions.ApiError as e:
        print(f"Error processing place_id {i}: {e}")
        continue
    
    
    if gplaces_detail['status'] == 'NOT_FOUND':
        print("NOT_FOUND")
        continue
    
    if gplaces_detail['status'] != 'OK':
        print(gplaces_detail['status'])
        break
    
    result = gplaces_detail['result']
    
    # 将结果添加到detail_data DataFrame
    new_row = {
        "place_id": i,
        "formatted_phone_number": result.get("formatted_phone_number", ""),
        "international_phone_number": result.get("international_phone_number", ""),
        "website": result.get("website", ""),
        "types": ", ".join(result.get("types", []))
    }

    detail_data = pd.concat([detail_data, pd.DataFrame([new_row])], ignore_index=True)


Processing place_id ChIJm5GlEv1XkWsRvjOQPX3F3wA (1/587)
Processing place_id ChIJObxk2iG1lmsRME1sAOpIod0 (2/587)
Processing place_id ChIJqzMcVCdakWsRAdIdJ_g149w (3/587)
Processing place_id ChIJ14aDK2ho1moR-3SQwicF9Xc (4/587)
Processing place_id ChIJteXpk-9gwAoREQxxEnjZfkw (5/587)
Processing place_id ChIJcwQxp-RNkWsRTPeO7oENU-A (6/587)
Processing place_id ChIJAQDwppVZkWsRBKRz70-tGy8 (7/587)
Processing place_id ChIJeb5qCrJQkWsRkMpAOENE3bU (8/587)
Processing place_id ChIJQei6DbVQkWsRu60ld2Ya5mg (9/587)
Processing place_id ChIJLX4_UY5XkWsRxCexyVcB3Q0 (10/587)
Processing place_id ChIJQ5H0lp3jk2sRxOiUaoSqKP8 (11/587)
Processing place_id ChIJkx-XD6Djk2sRFI7hkJwd7Bw (12/587)
Processing place_id ChIJVwT-5gBXkWsRw7XfIIUj5N4 (13/587)
Processing place_id ChIJ9bKd1SVfkWsRrspe9oJGdKY (14/587)
Processing place_id ChIJUXCBcgfjk2sRoYdhFTNjAw8 (15/587)
Processing place_id ChIJ0Z9G1D1HkWsRotkaL4iTTZ8 (16/587)
Processing place_id ChIJC6b6OFRNkWsRzE6LHLLYpKk (17/587)
Processing place_id ChIJ7TNWLpnjk2sRkhf6

In [41]:
# 合併表格
final_merge = detail_data.merge(final_data, on='place_id', how='left')

final_merge

,place_id,formatted_phone_number,international_phone_number,website,types_x,name,formatted_address,business_status,rating,reference,types_y,user_ratings_total,keywords,permanently_closed,price_level,Potential_Client
0,ChIJm5GlEv1XkWsRvjOQPX3F3wA,0483 083 201,+61 483 083 201,https://4ahaus.com/,"general_contractor, point_of_interest, establi...",4aHAUS Design Studio,"7 Ashgrove Ave, Ashgrove QLD 4060, Australia",OPERATIONAL,5.0,ChIJm5GlEv1XkWsRvjOQPX3F3wA,"['general_contractor', 'point_of_interest', 'e...",9,Interior Design,NaN,NaN,Y
1,ChIJObxk2iG1lmsRME1sAOpIod0,0488 770 607,+61 488 770 607,http://www.ellenbe.com.au/,"general_contractor, point_of_interest, establi...",Ellenbe Interior Design,"16 Tetra Cres, Ripley QLD 4306, Australia",OPERATIONAL,0.0,ChIJObxk2iG1lmsRME1sAOpIod0,"['general_contractor', 'point_of_interest', 'e...",0,Interior Design,NaN,NaN,NaN
2,ChIJqzMcVCdakWsRAdIdJ_g149w,0418 775 714,+61 418 775 714,http://fairweather.net.au/,"point_of_interest, establishment",Fairweather Architecture,"6 Amity St, New Farm QLD 4005, Australia",OPERATIONAL,0.0,ChIJqzMcVCdakWsRAdIdJ_g149w,"['point_of_interest', 'establishment']",0,Interior Design,NaN,NaN,NaN
3,ChIJ14aDK2ho1moR-3SQwicF9Xc,1300 029 816,+61 1300 029 816,http://centre-square.com.au/,"general_contractor, point_of_interest, establi...",Centresquare,"49 Corymbia Cres, Anstead QLD 4070, Australia",OPERATIONAL,4.7,ChIJ14aDK2ho1moR-3SQwicF9Xc,"['general_contractor', 'point_of_interest', 'e...",24,Interior Design,NaN,NaN,NaN
4,ChIJteXpk-9gwAoREQxxEnjZfkw,0450 330 413,+61 450 330 413,http://www.janellemeadinteriors.com.au/,"general_contractor, point_of_interest, establi...",Janelle Mead Interiors,"2/830 Old Cleveland Rd, Carina QLD 4152, Austr...",OPERATIONAL,5.0,ChIJteXpk-9gwAoREQxxEnjZfkw,"['general_contractor', 'point_of_interest', 'e...",1,Interior Design,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582,ChIJ7WXV0_1vkWsRdE6umZBL-TA,0499 363 000,+61 499 363 000,,"point_of_interest, store, establishment","Russell Island Timber, Decking & Cladding","14 Trimaran St, Russell Island QLD 4184, Austr...",OPERATIONAL,4.8,ChIJ7WXV0_1vkWsRdE6umZBL-TA,"['store', 'point_of_interest', 'establishment']",4,Renovation,NaN,NaN,NaN
583,ChIJpWn-YaBvkWsRhpIhTPEZa_s,0466 890 266,+61 466 890 266,https://www.mpii.com.au/,"general_contractor, point_of_interest, establi...",FALCON CASTLE,"10 Falconhurst Rd, Russell Island QLD 4184, Au...",OPERATIONAL,5.0,ChIJpWn-YaBvkWsRhpIhTPEZa_s,"['general_contractor', 'point_of_interest', 'e...",1,Renovation,NaN,NaN,NaN
584,ChIJUeQfkYBDkWsRFvOCbb3YbS8,1300 166 209,+61 1300 166 209,https://briscoroofing.net.au/,"roofing_contractor, point_of_interest, establi...",Brisco Roofing,"58 Belford Dr, Wellington Point QLD 4160, Aust...",OPERATIONAL,4.9,ChIJUeQfkYBDkWsRFvOCbb3YbS8,"['roofing_contractor', 'point_of_interest', 'e...",130,Renovation,NaN,NaN,NaN
585,ChIJcyA2kMVhkWsRm6Aj1JOnhNQ,0437 375 827,+61 437 375 827,http://www.juststripit.com.au/,"general_contractor, point_of_interest, establi...",Just Strip It,"Unit 16/15 Peterson St, Wellington Point QLD 4...",OPERATIONAL,5.0,ChIJcyA2kMVhkWsRm6Aj1JOnhNQ,"['general_contractor', 'point_of_interest', 'e...",2,Renovation,NaN,NaN,NaN


In [42]:
# 電話去空格
final_merge['formatted_phone_number'] = final_merge['formatted_phone_number'].str.replace(' ', '', regex=True)

final_merge['international_phone_number'] = final_merge['international_phone_number'].str.replace(' ', '', regex=True)

# 存為 excel 之前檢查資料筆數
print("Before saving to Excel:", len(final_merge))

Before saving to Excel: 587


In [43]:
final_merge.to_excel("AU_data_excluded_20250314(直客).xlsx", index=False)

# 跟CRM去重

In [44]:
# 取得 Token , 設置請求參數 (台灣)
url = "https://login-p10.xiaoshouyi.com/auc/oauth2/token"
headers = {
    "Content-Type": "application/x-www-form-urlencoded"
}
data = {
    "grant_type": "password",
    "client_id": "client_id",
    "client_secret": "client_secret",
    "redirect_uri": "https://api-p10.xiaoshouyi.com",
    "username": "11021041@twkd.com",
    "password": "password"
}

# 發送 post 請求
response = req.post(url, headers=headers, data=data)

if response.status_code == 200:
    print(response.json())
else:
    print("請求失敗, 狀態碼:", response.status_code)

content = response.json()
ac_token = content["access_token"]


print(ac_token)

{'access_token': '417c561fa8e455dc859a9396c08353582655ff4631297683d401fafa289c9d35.MjU0NDU2OTY4NDAzNzYxMA==9', 'token_type': 'Bearer', 'expires_in': 86399, 'scope': 'all prm crm', 'tenant_id': '2544569684037610', 'login_type': '', 'encryption_key': '', 'client_id': 'bc4896f9e3e4fad682f9fe60d5fbaa2e', 'passport_id': '3006921588936589', 'mobile_params': '', 'instance_uri': 'api-p10.xiaoshouyi.com'}
417c561fa8e455dc859a9396c08353582655ff4631297683d401fafa289c9d35.MjU0NDU2OTY4NDAzNzYxMA==9


In [45]:
#### Company #### 
url_2 = "https://api-p10.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type": "application/x-www-form-urlencoded"
}
queryLocator = ''
crm_data = pd.DataFrame()

while True:
    data = {
        "xoql": '''SELECT 
        dimDepart.departName as 資料區域群組,
        customItem198__c.name as 公司型態代號,
        accountCode__c as CompanyID,
        accountName as 公司全名,
        customItem197__c as 公司簡稱,
        customItem202__c as 公司地址,
        Phone__c as combined_phone,
        SAP_CompanyID__c as SAP公司代號,
        contactNumber__c as 主要聯絡人行動電話
        FROM account 
        WHERE dimDepart.departName LIKE '%AU%'
        ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = req.post(url_2, headers=headers, data=data)
    exist = response.json()
    data = pd.DataFrame(exist["data"]["records"])
    crm_data = pd.concat([crm_data, data], ignore_index=True, sort=False)

    if not exist['queryLocator']:
        break
    queryLocator = exist['queryLocator']

crm_data

,資料區域群組,公司型態代號,CompanyID,公司全名,公司簡稱,公司地址,combined_phone,SAP公司代號,主要聯絡人行動電話
0,AU,IB,GAC00093798,Indent Tile Centre,Indent Tile Centre,"11 Grey Gum Rd, Taree NSW 2430, Australia",0265524688,,
1,AU,IB,GAC00093799,Northern Rivers Tiles & Mosaics,Northern Rivers Tile,"177 Keen St, Lismore NSW 2480, Australia",0266215232,,
2,AU,IB,GAC00093800,Color Craft Kitchens,Color Craft Kitchens,"125 Mount Darragh Rd, Pambula NSW 2549, Australia",0264957153,,
3,AU,IB,GAC00093801,Stone Ambassador Western Australia,Stone Ambassador,"24 Creative St, Wangara WA 6065, Australia",0863836558,,
4,AU,IB,GAC00093802,Cabinet Makers Geelong,Geelong,"7 hardwood walk, leopold, vic 3224",0488860284,,610420940514
...,...,...,...,...,...,...,...,...,...
465,AU-加盟,N,GAC00167120,BhoomiPatel,BhoomiPatel,,,,
466,AU-加盟,,GAC00167128,p L,p L,,,,
467,AU-加盟,,GAC00167138,Arvant Solutions,Arvant Solutions,,,,
468,AU-加盟,,GAC00167512,Gora logistics pty Ltd,Gora logistics pty Ltd,,,,


In [ ]:
# 如果combined_phone為空，則使用主要聯絡人行動電話
crm_data['combined_phone'].fillna(crm_data['主要聯絡人行動電話'], inplace=True)

# 去掉CRM_data中combined_phone中的標點符號和空格
crm_data['new_phone_num'] = crm_data['combined_phone'].str.replace(r'\D', '', regex=True)

# 將na補0
crm_data['new_phone_num'].fillna("0", inplace=True)

# 只取 final_merge['formatted_phone_number']的數字部分
final_merge['new_phone_num'] = final_merge['formatted_phone_number'].str.replace(r'\D', '', regex=True)

# 選擇需要的欄位
req_columns_final_merge = ["place_id", "name", "formatted_address", "new_phone_num"]
final_merge = final_merge[req_columns_final_merge]
final_merge.columns = ["place_id", "name", "formatted_address", "new_phone_num"]

req_columns_crm_data = ["公司全名", "CompanyID", "公司地址", "new_phone_num"]
crm_data = crm_data[req_columns_crm_data]
crm_data.columns = ["name", "CompanyID", "formatted_address", "new_phone_num"]

# name exact first
final_merge = final_merge[~final_merge['name'].isin(crm_data['name'])]

# exact address match
final_merge = final_merge[~final_merge['formatted_address'].isin(crm_data['formatted_address'])]

# na_phone_crm_data
na_phone_crm_data = crm_data[~crm_data['new_phone_num'].isna()]

# exact phone match
exact_phone_data = pd.merge(final_merge, na_phone_crm_data, left_on="new_phone_num", right_on="new_phone_num", how="inner")
final_merge = final_merge[~final_merge['place_id'].isin(exact_phone_data['place_id'])]


final_merge

,place_id,name,formatted_address,new_phone_num
0,ChIJm5GlEv1XkWsRvjOQPX3F3wA,4aHAUS Design Studio,"7 Ashgrove Ave, Ashgrove QLD 4060, Australia",0483083201
1,ChIJObxk2iG1lmsRME1sAOpIod0,Ellenbe Interior Design,"16 Tetra Cres, Ripley QLD 4306, Australia",0488770607
2,ChIJqzMcVCdakWsRAdIdJ_g149w,Fairweather Architecture,"6 Amity St, New Farm QLD 4005, Australia",0418775714
3,ChIJ14aDK2ho1moR-3SQwicF9Xc,Centresquare,"49 Corymbia Cres, Anstead QLD 4070, Australia",1300029816
4,ChIJteXpk-9gwAoREQxxEnjZfkw,Janelle Mead Interiors,"2/830 Old Cleveland Rd, Carina QLD 4152, Austr...",0450330413
...,...,...,...,...
582,ChIJ7WXV0_1vkWsRdE6umZBL-TA,"Russell Island Timber, Decking & Cladding","14 Trimaran St, Russell Island QLD 4184, Austr...",0499363000
583,ChIJpWn-YaBvkWsRhpIhTPEZa_s,FALCON CASTLE,"10 Falconhurst Rd, Russell Island QLD 4184, Au...",0466890266
584,ChIJUeQfkYBDkWsRFvOCbb3YbS8,Brisco Roofing,"58 Belford Dr, Wellington Point QLD 4160, Aust...",1300166209
585,ChIJcyA2kMVhkWsRm6Aj1JOnhNQ,Just Strip It,"Unit 16/15 Peterson St, Wellington Point QLD 4...",0437375827


In [47]:
original_data = pd.read_excel('AU_data_excluded_20250314(直客).xlsx', dtype=str)

# 將 original_data 與 final_merge 以 place_id 欄位進行合併
merged_data = original_data.merge(final_merge, on='place_id', how='inner')

merged_data = merged_data.rename(columns={'name_x': 'name'})

merged_data = merged_data.drop_duplicates(subset='name', keep='first')

merged_data = merged_data.drop('name_y', axis=1)

merged_data = merged_data.drop_duplicates()

merged_data = merged_data.dropna(subset=['formatted_phone_number', 'international_phone_number', 'new_phone_num'])

merged_data

,place_id,formatted_phone_number,international_phone_number,website,types_x,name,formatted_address_x,business_status,rating,reference,types_y,user_ratings_total,keywords,permanently_closed,price_level,Potential_Client,formatted_address_y,new_phone_num
0,ChIJm5GlEv1XkWsRvjOQPX3F3wA,0483083201,+61483083201,https://4ahaus.com/,"general_contractor, point_of_interest, establi...",4aHAUS Design Studio,"7 Ashgrove Ave, Ashgrove QLD 4060, Australia",OPERATIONAL,5,ChIJm5GlEv1XkWsRvjOQPX3F3wA,"['general_contractor', 'point_of_interest', 'e...",9,Interior Design,NaN,NaN,Y,"7 Ashgrove Ave, Ashgrove QLD 4060, Australia",0483083201
1,ChIJObxk2iG1lmsRME1sAOpIod0,0488770607,+61488770607,http://www.ellenbe.com.au/,"general_contractor, point_of_interest, establi...",Ellenbe Interior Design,"16 Tetra Cres, Ripley QLD 4306, Australia",OPERATIONAL,0,ChIJObxk2iG1lmsRME1sAOpIod0,"['general_contractor', 'point_of_interest', 'e...",0,Interior Design,NaN,NaN,NaN,"16 Tetra Cres, Ripley QLD 4306, Australia",0488770607
2,ChIJqzMcVCdakWsRAdIdJ_g149w,0418775714,+61418775714,http://fairweather.net.au/,"point_of_interest, establishment",Fairweather Architecture,"6 Amity St, New Farm QLD 4005, Australia",OPERATIONAL,0,ChIJqzMcVCdakWsRAdIdJ_g149w,"['point_of_interest', 'establishment']",0,Interior Design,NaN,NaN,NaN,"6 Amity St, New Farm QLD 4005, Australia",0418775714
3,ChIJ14aDK2ho1moR-3SQwicF9Xc,1300029816,+611300029816,http://centre-square.com.au/,"general_contractor, point_of_interest, establi...",Centresquare,"49 Corymbia Cres, Anstead QLD 4070, Australia",OPERATIONAL,4.7,ChIJ14aDK2ho1moR-3SQwicF9Xc,"['general_contractor', 'point_of_interest', 'e...",24,Interior Design,NaN,NaN,NaN,"49 Corymbia Cres, Anstead QLD 4070, Australia",1300029816
4,ChIJteXpk-9gwAoREQxxEnjZfkw,0450330413,+61450330413,http://www.janellemeadinteriors.com.au/,"general_contractor, point_of_interest, establi...",Janelle Mead Interiors,"2/830 Old Cleveland Rd, Carina QLD 4152, Austr...",OPERATIONAL,5,ChIJteXpk-9gwAoREQxxEnjZfkw,"['general_contractor', 'point_of_interest', 'e...",1,Interior Design,NaN,NaN,NaN,"2/830 Old Cleveland Rd, Carina QLD 4152, Austr...",0450330413
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
556,ChIJ7WXV0_1vkWsRdE6umZBL-TA,0499363000,+61499363000,NaN,"point_of_interest, store, establishment","Russell Island Timber, Decking & Cladding","14 Trimaran St, Russell Island QLD 4184, Austr...",OPERATIONAL,4.8,ChIJ7WXV0_1vkWsRdE6umZBL-TA,"['store', 'point_of_interest', 'establishment']",4,Renovation,NaN,NaN,NaN,"14 Trimaran St, Russell Island QLD 4184, Austr...",0499363000
557,ChIJpWn-YaBvkWsRhpIhTPEZa_s,0466890266,+61466890266,https://www.mpii.com.au/,"general_contractor, point_of_interest, establi...",FALCON CASTLE,"10 Falconhurst Rd, Russell Island QLD 4184, Au...",OPERATIONAL,5,ChIJpWn-YaBvkWsRhpIhTPEZa_s,"['general_contractor', 'point_of_interest', 'e...",1,Renovation,NaN,NaN,NaN,"10 Falconhurst Rd, Russell Island QLD 4184, Au...",0466890266
558,ChIJUeQfkYBDkWsRFvOCbb3YbS8,1300166209,+611300166209,https://briscoroofing.net.au/,"roofing_contractor, point_of_interest, establi...",Brisco Roofing,"58 Belford Dr, Wellington Point QLD 4160, Aust...",OPERATIONAL,4.9,ChIJUeQfkYBDkWsRFvOCbb3YbS8,"['roofing_contractor', 'point_of_interest', 'e...",130,Renovation,NaN,NaN,NaN,"58 Belford Dr, Wellington Point QLD 4160, Aust...",1300166209
559,ChIJcyA2kMVhkWsRm6Aj1JOnhNQ,0437375827,+61437375827,http://www.juststripit.com.au/,"general_contractor, point_of_interest, establi...",Just Strip It,"Unit 16/15 Peterson St, Wellington Point QLD 4...",OPERATIONAL,5,ChIJcyA2kMVhkWsRm6Aj1JOnhNQ,"['general_contractor', 'point_of_interest', 'e...",2,Renovation,NaN,NaN,NaN,"Unit 16/15 Peterson St, Wellington Point QLD 4...",0437375827


In [48]:
merged_data.to_excel('AU_data_excluded_20250314(直客)_final.xlsx', index=False)

In [ ]:
import openai 
import pandas as pd
import json

# === 1. 讀取店家資料 ===
df = pd.read_excel("../AU_header_C&D_20250313_excluded.xlsx") 

# === 2. 將 DataFrame 轉成 JSON 傳給 Azure OpenAI ===
json_data = df.to_json(orient="records", force_ascii=False) 

endpoint = os.getenv("ENDPOINT_URL", )  
deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4")  
subscription_key = os.getenv("AZURE_OPENAI_API_KEY", )  

# 使用金鑰型驗證，將 Azure OpenAI 服務用戶端初始化    
client = openai.AzureOpenAI(  
    azure_endpoint=endpoint,  
    api_key=subscription_key,  
    api_version="2024-05-01-preview",
)

# 目標區域
location = 'Logan/Brisbane/Redland, Australia'

# 設定 OpenAI 請求
def filter_places(business):
    prompt = f"""
    這是一份 Google Maps API 抓取的店家資料，請幫我篩選符合條件的店家。
    條件如下：
    1.這間店是否與 "Interior Design"（室內設計）相關？
    2.這間店是否提供室內設計、裝潢、空間規劃、住宅或商業室內設計服務?
    3.這間店的地址是否在{location}?

    以下是店家清單：
    {business}

    **請務必僅輸出 JSON 格式，不要添加任何解釋或說明。**
    """

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "system", "content": "你是一位資料篩選專家，請根據條件篩選店家"},
                  {"role": "user", "content": prompt}],
        temperature=0.5
    )

    return response.choices[0].message.content.strip()  

# 篩選符合條件的店家
filtered_places = filter_places(json_data)
print(filtered_places)

# 解析 JSON
try:
    filtered_data = json.loads(filtered_places)
except json.JSONDecodeError:
    print("❌ JSON 解析失敗，輸出內容:", filtered_places)
    filtered_data = []  # 設定為空列表，以防止後續 DataFrame 出錯

df = pd.DataFrame(filtered_data)

df

[
    {
        "place_id": "ChIJm5GlEv1XkWsRvjOQPX3F3wA",
        "name": "4aHAUS Design Studio",
        "formatted_address": "7 Ashgrove Ave, Ashgrove QLD 4060, Australia",
        "business_status": "OPERATIONAL",
        "rating": 5.0,
        "reference": "ChIJm5GlEv1XkWsRvjOQPX3F3wA",
        "types": "['general_contractor', 'point_of_interest', 'establishment']",
        "user_ratings_total": 9,
        "keywords": "Interior Design",
        "permanently_closed": null,
        "price_level": null
    },
    {
        "place_id": "ChIJObxk2iG1lmsRME1sAOpIod0",
        "name": "Ellenbe Interior Design",
        "formatted_address": "16 Tetra Cres, Ripley QLD 4306, Australia",
        "business_status": "OPERATIONAL",
        "rating": 0.0,
        "reference": "ChIJObxk2iG1lmsRME1sAOpIod0",
        "types": "['general_contractor', 'point_of_interest', 'establishment']",
        "user_ratings_total": 0,
        "keywords": "Interior Design",
        "permanently_closed": null,
   

,place_id,name,formatted_address,business_status,rating,reference,types,user_ratings_total,keywords,permanently_closed,price_level
0,ChIJm5GlEv1XkWsRvjOQPX3F3wA,4aHAUS Design Studio,"7 Ashgrove Ave, Ashgrove QLD 4060, Australia",OPERATIONAL,5.0,ChIJm5GlEv1XkWsRvjOQPX3F3wA,"['general_contractor', 'point_of_interest', 'e...",9,Interior Design,None,None
1,ChIJObxk2iG1lmsRME1sAOpIod0,Ellenbe Interior Design,"16 Tetra Cres, Ripley QLD 4306, Australia",OPERATIONAL,0.0,ChIJObxk2iG1lmsRME1sAOpIod0,"['general_contractor', 'point_of_interest', 'e...",0,Interior Design,None,None
2,ChIJqzMcVCdakWsRAdIdJ_g149w,Fairweather Architecture,"6 Amity St, New Farm QLD 4005, Australia",OPERATIONAL,0.0,ChIJqzMcVCdakWsRAdIdJ_g149w,"['point_of_interest', 'establishment']",0,Interior Design,None,None
